## Chapter 4　言語処理特有の深層学習の発展

---
### 4.1　注意機構
複数のベクトルがあるとき、どのベクトルを重要視するかも含めて学習させる仕組みを  
注意機構（attention mechanism）、あるいは注意と呼ぶ。

このうち、複数ベクトルの重み付き平均を使う方法をソフト注意機構（soft attention mechanism）と呼ぶ。

---
#### ソフト注意機構
長さ $I$ の入力系列 $\{\mathbf{x}_{1},\ldots,\mathbf{x}_{I}\}$ に対し、各時刻で符号化されたベクトルを $\{ \mathbf{h}_{1},\ldots,\mathbf{h}_{I}\}$ とおく。  
この計算は、RNN の遷移関数を $\Psi^{(s)}$ とするとき次の式による。

\begin{align*}
\mathbf{h}_{i}^{(s)} = \Psi^{(s)} ( \mathbf{x}_{i}, \mathbf{h}_{i-1}^{(s)})
\end{align*}

通常の系列変換モデルでは最後の状態出力 $\mathbf{h}_{I}$ のみを用いるが、
ここではより直接的に情報を伝播する方法を考える。

復号化器でも RNN により隠れ状態ベクトル $\mathbf{h}_{j}^{(t)}$ を計算する。

\begin{align*}
\mathbf{h}_{j}^{(t)} = \Psi^{(t)}(\mathbf{y}_{j}, \mathbf{h}_{j-1}^{(t)})
\end{align*}

いま、復号化器が $j$ 番目の単語を推定するときに、  
復号化器の $i$ 番目の隠れ状態ベクトル $\mathbf{h}_{i}^{(s)}$ の重要度を示すスカラー値の重みを $a_{i} \in \mathbb{R}$ とする。  
$a_{i}$ による $\mathbf{h}_{i}^{(s)}$ の重み付き平均 $\bar{\mathbf{h}}$ を

\begin{align*}
\bar{\mathbf{h}} = \sum_{\tilde{i}=1}^{I} a_{\tilde{i}}\mathbf{h}_{\tilde{i}^{(s)}}
\end{align*}

とする。

復号化器が $j$ 番目の単語の予測に $\bar{\mathbf{h}}$ を利用するため、$\mathbf{h}_{j}^{(t)}$ の代わりに次式で定義される $\hat{\mathbf{h}}_{j}^{(t)}$ を利用する。

\begin{align*}
\hat{\mathbf{h}}_{j}^{(t)} = \operatorname{tanh} \left( \mathbf{W}^{(a)} \left[ \begin{array} \,\bar{\mathbf{h}}\\ \mathbf{h}_{j}^{(t)} \end{array} \right] \right)
\end{align*}

$\{a_{1},\ldots,a_{I}\}$ もモデル内で計算を行う。  
まず関数 $\Omega$ によって $\mathbf{h}_{i}^{(s)}$ と $\mathbf{h}_{j}^{(t)}$ の間の重み $e_{i}$ を計算する。

\begin{align*}
e_{i} = \Omega(\mathbf{h}_{i}^{(s)}, \mathbf{h}_{j}^{(t)})
\end{align*}

次に、$a_{i}$ は総和が１になるようにソフトマックス関数で正規化する。

\begin{align*}
a_{i} = \frac{\exp(e_{i})}{\sum_{\tilde{i}=1}^{I} \exp (e_{\tilde{i}})}
\end{align*}

上で使用した関数 $\Omega$ については様々な関数が利用されており、以下は一例。

\begin{align*}
\Omega (\mathbf{h}_{i}^{(s)}, \mathbf{h}_{j}^{(t)}) = \begin{cases}
\mathbf{h}_{i} \cdot \mathbf{h}_{j}^{(t)} \\
\mathbf{h}_{i}^{(s)} \cdot \mathbf{Wh}_{j}^{(t)} \\
\mathbf{v} \cdot \operatorname{tanh} \left( \mathbf{W} \left[ \begin{array}\, \mathbf{h}_{i}^{(s)}\\\mathbf{h}_{j}^{(t)} \end{array} \right] \right)
\end{cases}
\end{align*}

$a_{i}$ は確率として解釈することもでき、  
このとき $\bar{\mathbf{h}}$ は $\{a_{1},\ldots,a_{I}\}$ の分布における $\mathbf{h}_{i}^{(s)}$ の期待値と見なせる。

より一般化する。  
$N$ 個の参照したい情報 $\mathbf{Y} = \{ \mathbf{y}_{1}, \ldots, \mathbf{y}_{N}\}$ （上の例では符号化器の隠れ状態）があり、  
それぞれの重要度、あるいはいずれかが選択される確率 $\{ a_{1},\ldots,a_{N}\}$ を計算する。  
$a_{i}$ の計算については何らかの情報 $\mathbf{c}_{i}$ を用いる（上の例では隠れ状態）。

例えば end-to-end 記憶ネットワークでは記憶を参照するのにソフト注意機構を利用しており

\begin{align*}
a_{i} = \frac{\exp(\Omega(\mathbf{c}_{i}))}{\sum_{i=1}^{I} \exp (\Omega (\mathbf{c}_{\tilde{i}}))}
\end{align*}

を利用して次の通り $\hat{\mathbf{y}}$ を得る。

\begin{align*}
\hat{\mathbf{y}} = \mathbb{E}_{a}[\mathbf{y}] = \sum_{i=1}^{N} a_{i}\mathbf{y}_{i}
\end{align*}



---
#### ハード注意機構
ソフト注意機構では複数の情報源が選択される確率に対しその情報の期待値を確定的に計算したのに対し、  
ハード注意機構では情報源の中から確率的に１つ選択されるものとして計算する。

$N$ 個の参照したい情報 $\mathbf{Y} = \{ \mathbf{y}_{1}, \ldots, \mathbf{y}_{N}\}$ があり、  
確率変数 $X$ が $\{1,\ldots,N\}$ の中から確率的に１つを選択するものとして  
この時の確率を $P(X=i) = a_{i}$ とおく。すなわち

\begin{align*}
P(X=i)=a_{i}
\end{align*}

である。$\mathbf{Y}$ の中から $X$ を使って無作為に１つ選択された結果を

\begin{align*}
\hat{\mathbf{y}} = \mathbf{y}X
\end{align*}

とする。

ハード注意機構を利用した学習で目的関数 $f(\hat{\mathbf{y}})$ が計算されたとして、これを最小化する。  
$\hat{\mathbf{y}}$ および $f(\hat{\mathbf{y}})$ は確率的に振る舞うため、期待値 $\mathbb{E}[f(\hat{\mathbf{y}})]$ を最小化の対象とする。  
期待値の微分は次の通り。

\begin{align*}
\nabla \mathbb{E}[f(\hat{\mathbf{y}})] &= \nabla \sum_{x=1}^{N} f( \mathbf{y}_{x}) a_{x} \\
&= \sum_{x=1}^{N} \nabla f(\mathbf{y}_{x}) a_{x} + \sum_{x=1}^{N} f(\mathbf{y}_{x}) \nabla a_{x} \\
&= \mathbb{E}[\nabla f(\hat{\mathbf{y}})] + \sum_{x=1}^{N} f(\mathbf{y}_{x}) \nabla a_{x}
\end{align*}

いずれの項もすべての $x$ について計算することから計算の負荷が高く、近似が必要となる。  
第１項についてはモンテカルロ法を用いる、つまり有限個の標本で次のように近似する。

\begin{align*}
\mathbb{E}[\nabla f(\hat{\mathbf{y}})] \approx \frac{1}{T}\sum_{i=1}^{T} \nabla f(\mathbf{y}_{\bar{x}_{i}})
\end{align*}

第２項についても変形すると期待値の式に変形できるので、モンテカルロ法が適用可能である。

\begin{align*}
\sum_{x=1}^{N} f (\mathbf{y}_{x}) \nabla a_{x} &= \sum_{x=1}^{N} f(\mathbf{y}_{x}) \nabla a_{x} \frac{a_{x}}{a_{x}} \\
&= \mathbb{E} \left[ f( \hat{\mathbf{y}} ) \frac{\nabla a_{x}}{a_{x}} \right] \\
&= \mathbb{E} [ f( \hat{\mathbf{y}}) \nabla \log a_{x}]
\end{align*}

以上により

\begin{align*}
\nabla \mathbb{E}[f(\hat{\mathbf{y}})] &= \mathbb{E}[\nabla f(\hat{\mathbf{y}}) + f( \hat{\mathbf{y}}) \nabla \log a_{x}] \\
&\approx \frac{1}{T}\sum_{i=1}^{T} \{ \nabla f(\mathbf{y}_{\bar{x}_{i}}) + f(\mathbf{y}_{\bar{x}_{i}}) \nabla \log a_{\bar{x}_{i}} \}
\end{align*}

となる。

なお実用上、標本から計算される $f( \mathbf{y}_{\bar{x}_{i}}) \nabla \log a_{\bar{x}_{i}}$ の分散が大きい時に学習できないことがある。  
そこで分散を抑えることを考える。定数 $b$ を用いて

\begin{align*}
\mathbb{e}[f( \hat{\mathbf{y}} ) \nabla \log a_{x}] &= \mathbb{E}[(f( \hat{\mathbf{y}} ) -b ) \nabla \log a_{x}] + \mathbb{E}[ b \nabla \log a_{x}] \\
&= \mathbb{E}[(f( \hat{\mathbf{y}} ) -b ) \nabla \log a_{x}] + b \mathbb{E}[\nabla \log a_{x}] \\
\mathbb{E}[\nabla \log a_{x}] &= \sum_{x=1}^{N} \nabla a_{x} = \nabla \sum_{x}^{N} a_{x}
\end{align*}

と変形する。$a_{x}$ は確率値であるから、$ \sum_{x=1}^{N}a_{x}=1, \nabla \sum_{x=1}^{N}a_{x}=0$ である。  
よって任意の定数 $b \in \mathbb{R}$ について

\begin{align*}
\mathbb{E}[f( \hat{\mathbf{y}} ) \nabla \log a_{x}] = \mathbb{E}[(f( \hat{\mathbf{y}} ) -b ) \nabla \log a_{x}]
\end{align*}

が成り立ち、この $b$ をベースラインと呼ぶ。  
ベースラインとしては $b=\mathbb{E}[f( \hat{\mathbf{y}})]$ がよく用いられる。

---
その他の注意機構としては、局所注意機構（local attention mechanism）などがある。

---
### 4.2　記憶ネットワーク
記憶ネットワーク（memory networks）は、記憶したい事例の書き込み・読み込み操作をモデル化したものである。

---
#### 記憶ネットワークのモデル
記憶ネットワークは $N$ 個の記憶情報 $\mathbf{M} = (\mathbf{m}_{i=1}^{N})$ を列として持ち、  
内部構造は $I,G,O,R$ ４つの部品に分けてモデル化されている。
 - Input feature map：入力情報変換、入力された情報を内部表現に変換する
 - Genelarization：一般化、新しい情報により内部の記憶情報を更新する
 - Output feature map：出力情報変換、外からの質問に対し記憶情報を利用して返答のための内部表現を生成する
 - Response：応答、出力情報を文など適切な返答に変換する
 
これらは符号化復号化モデルに記憶を取り扱う部分を追加したものと見なすことができる。

---
#### 教師あり記憶ネットワーク
最も簡単な状況について考察するため、記憶情報・入力・返答に加えて  
返答生成のためにどの記憶情報を利用すべきかという根拠情報（supporting fact）が揃っていることを仮定する。

このようなモデルは、根拠情報を与えることを明示するために強教師あり記憶ネットワークとも呼ばれる。

いま、入力が単語列、出力（返答）が単語であるシステムを考え、  
入力は $D$ 次元ベクトルの内部情報に変換され、記憶情報の各要素は $\mathbf{m}_{i} \in \mathbb{R}^{D}$ とする。

入力情報変換処理は埋め込みベクトルの和の形への変換とする。

\begin{align*}
I(\mathbf{x}) = \sum \mathbf{Ex}
\end{align*}

一般化処理は次の通り。

\begin{align*}
\mathbf{m}_{N} \leftarrow \mathbf{x}
\end{align*}

出力情報変換について考える。  
返答するのに必要な情報か否かを判定するスコア関数があるとして、これを $s_{O}$ で表す。  
入力 $\mathbf{x} \in \mathbb{R}^{D}$ に対してスコア上位 $\kappa$ を記憶 $\mathbf{M}$ から探す。  
ここでは $\kappa = 2$ として

\begin{align*}
o_{1} = O_{1}(\mathbf{x}, \mathbf{M}) &= \operatorname{argmax}_{i=1, \ldots, N} s_{O} (\mathbf{x}, \mathbf{m}_{i}) \\
o_{2} = O_{2}(\mathbf{x}, \mathbf{M}) &= \operatorname{argmax}_{i=1, \ldots, N} s_{O} ((\mathbf{x}, \mathbf{m}_{o_{1}}), \mathbf{m}_{i})
\end{align*}

ただし $s_{O}$ は複数の文書を扱えるような第一引数をとれるものとする。  

返答は $\mathbf{x}, \mathbf{m}_{1}, \mathbf{m}_{2}$ の３つの情報から生成する。  
ここでは問題設定より単語を１つ答えるので、語彙集合 $\mathcal{V}$ の中からスコア $s_{R}$ が最大となる単語を探す。

\begin{align*}
r = \operatorname{argmax}_{w \in \mathcal{V}} s_{r} ((\mathbf{x}, \mathbf{m}_{o_{1}}, \mathbf{m}_{o_{2}}),w)
\end{align*}

２つのスコア関数 $s_{O}, s_{R}$ は、パラメータ $\mathbf{U} \in R_{N \times D}$ を用いて次の形式で表せる。

\begin{align*}
s(\mathbf{x}, \mathbf{y}) = \Phi^{(x)}(\mathbf{x})^{\mathrm{T}}\mathbf{U}^{\mathrm{T}}\mathbf{U} \Phi^{(y)}(\mathbf{y})
\end{align*}

ここで $\Phi$ は入力情報を $D$ 次元ベクトルの特徴ベクトルに変換する関数であり、  
種々の方法があるが、最も単純なものは単語の埋め込みベクトルの和である。  
実装時には $s_{O},s_{R}$ で異なるパラメータを用い、$\mathbf{x},\mathbf{m}_{o_{1}},\mathbf{m}_{o_{2}}$ のそれぞれに対し別の埋め込みベクトルを用いる。

最大値を取るべき候補が $o_{1}^{*}, o{2}^{*},r^{*}$ として与えられたものとする。  
学習時は、以下の式を最小化する。

\begin{align*}
\sum_{o \in \{ 1, \ldots, N \} \backslash \{ o_{1}^{*} \} } &\max (0, \gamma - s_{O}(\mathbf{x}, \mathbf{m}_{o_{1}^{*}}) + s_{O}(\mathbf{x},\mathbf{m}_{O})) \\
+ \sum_{o \in \{ 1, \ldots, N \} \backslash \{ o_{1}^{*} \} } &\max (0, \gamma - s_{O}((\mathbf{x}, \mathbf{m}_{o_{1}^{*}}),\mathbf{m}_{o_{2}^{*}}) + s_{O}((\mathbf{x}, \mathbf{m}_{o_{1}^{*}}),\mathbf{m}_{o})) \\
+ \sum_{o \in \mathcal{V} \backslash \{ r^{*} \} } &\max (0, \gamma - s_{O}((\mathbf{x}, \mathbf{m}_{o_{1}^{*}},\mathbf{m}_{o_{2}^{*}}),r^{*}) + s_{O}((\mathbf{x}, \mathbf{m}_{o_{1}^{*}},\mathbf{m}_{o_{2}^{*}}),r))
\end{align*}

このようなネットワーク用のタスクセットとしては、例えばbAbIタスクがある。

---
#### end-to-end 記憶ネットワーク
現実的な質疑応答タスクでは、  
先の教師あり記憶ネットワークで用いたような「どの記憶を参照するか」という中間的な部分課題の解が  
明示的に得られず、訓練データに含めることが不可能な場合が多い。  

記憶ネットワークの全ての部品を微分可能な関数として記述し  
中間的な部分課題の解を用いずに一気に学習させる手法を end-to-end 記憶ネットワークと呼ぶ。  
ここでは根拠情報の代わりに注意機構を利用して情報を引き出すこととする。

入力情報変換では、入力文 $\mathbf{x}_{i}$ を $D \times |\mathcal{V}|$ の埋め込み行列 $\mathbf{A} \in \mathbb{R}^{D \times \mathcal{V}}$ を使って、次の単一ベクトル $\mathbf{m}_{i}$ に符号化する。

\begin{align*}
\mathbf{m}_{i} = \sum_{j} \mathbf{Ax}_{ij}
\end{align*}

同様に、質問文 $q$ を $D \times |\mathcal{V}|$ の埋め込み行列 $\mathbf{B} \in \mathbb{R}^{D \times \mathcal{V}}$ を使って、次の単一ベクトル $\mathbf{u}$ に符号化する。

\begin{align*}
\mathbf{u} = \sum_{j} \mathbf{Bq}_{j}
\end{align*}

記憶情報の参照では、  
まず $N$ 個の記憶情報 $\{ \mathbf{m}_{1}, \ldots, \mathbf{m}_{N}\}$ のそれぞれに対して、$\mathbf{u}$ にとっての重要度 $\{ p_{1}, \ldots, p_{N}\}$ を計算する。  
これは内積のソフトマックスにより、次式の通り。

\begin{align*}
p_{i} = \frac{\exp ( \mathbf{u} \cdot \mathbf{m}_{i})}{\sum_{j=1}^{N} \exp ( \mathbf{u} \cdot \mathbf{m}_{j})}
\end{align*}

回答に利用する出力の情報では、  
次の式によって出力情報 $\mathbf{c}_{i}$ をつくり、全体の出力情報 $\mathbf{o} \in \mathbb{R}^{D}$ を求める。

\begin{align*}
\mathbf{c}_{i} &= \sum_{j} \mathbf{C} \mathbf{x}_{ij} \\
\mathbf{o} &= \sum_{i=1}^{N} p_{i} \mathbf{c}_{i}
\end{align*}

最後に $\mathbf{u}$ と $\mathbf{o}$、$|\mathcal{V}| \times D$ 行列 $\mathbf{W} \in \mathbb{R}^{|\mathcal{V} \times D}$ を用いて各返答単語に対するスコアを計算し、全体で１になるよう正規化する。

\begin{align*}
\hat{\mathbf{a}} = \operatorname{softmax} ( \mathbf{W}(\mathbf{o} + \mathbf{u}))
\end{align*}

以上の処理は微分可能な関数のみで構成されているので、  
訓練データが入力情報 $\{ \mathbf{x}_{i} \} _{i=1}^{N}$、質問文 $\mathbf{q}$、正しい返答 $a$ を含んでいれば  
$